In [1]:
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.sparse as sp

In [2]:
# from sklearn.naive_bayes import MultinomialNB - хуже работает
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [3]:
%%time
import pickle
with open('page_dict.p', 'rb') as fp:
    page_dict = pickle.load(fp)

CPU times: user 5.69 s, sys: 3.64 s, total: 9.33 s
Wall time: 11.9 s


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
train_data = pd.read_csv('train_groups.csv')
test_data = pd.read_csv('test_groups.csv')

In [6]:
# train_documents = []
# for i in train_data['doc_id']:
#     train_documents.append(page_dict.get(i))

In [7]:
# vectorizer = CountVectorizer(max_features=500)
# vectors = vectorizer.fit_transform(train_documents)

In [8]:
all_vectors_train = []
for group in tqdm_notebook(train_data.group_id.unique()):
    texts = []
    for i in train_data[train_data['group_id'] == group].doc_id:
        texts.append(page_dict.get(i))
    vectorizer = CountVectorizer(max_features=500)
    vectors = vectorizer.fit_transform(texts)
    all_vectors_train.append(vectors)

/Users/Denis/opt/anaconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [9]:
vectors_train = all_vectors_train[0]

In [10]:
for i in range(1, len(all_vectors_train)):
    vectors_train = sp.vstack((vectors_train, all_vectors_train[i]), format='csr')

In [11]:
vectors_train.shape

(11690, 500)

In [12]:
y_train = np.array(train_data['target'])

In [13]:
y_train.shape

(11690,)

In [14]:
clf = GaussianNB(var_smoothing=1e-12)

In [15]:
cross_val_score(clf, vectors_train.toarray(), y_train, cv=5, scoring='f1').mean()

0.5447029109791008

In [16]:
all_vectors_test = []
for group in tqdm_notebook(test_data.group_id.unique()):
    texts = []
    for i in test_data[test_data['group_id'] == group].doc_id:
        texts.append(page_dict.get(i))
    vectorizer = CountVectorizer(max_features=500)
    vectors = vectorizer.fit_transform(texts)
    all_vectors_test.append(vectors)

/Users/Denis/opt/anaconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [17]:
vectors_test = all_vectors_test[0]

In [18]:
for i in range(1, len(all_vectors_test)):
    vectors_test = sp.vstack((vectors_test, all_vectors_test[i]), format='csr')

In [19]:
clf = GaussianNB(var_smoothing=1e-12)

In [20]:
clf.fit(vectors_train.toarray(), y_train)

GaussianNB(priors=None, var_smoothing=1e-12)

In [21]:
test_data['target']  =  clf.predict(vectors_test.toarray())

In [22]:
test_data[['pair_id', 'target']].to_csv('try_naive.csv', index=None)